In [13]:

# from langchain_llm_test import ai_message
%pip install -qU python-dotenv langchain-upstage==0.5.0
%pip install --upgrade --quiet  docx2txt langchain-community langchain_pinecone
# %pip uninstall -y langchain-upstage langchain-openai langchain-core




[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-upstage 0.5.0 requires langchain-openai<0.3,>=0.2, but you have langchain-openai 0.3.34 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -qU langchain-text-splitters



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install -qU langchain-chroma


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install -qU langchain langchainhub



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader("./tax_docs/tax_with_table.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

In [10]:
len(document_list)

225

In [11]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()
embedding = UpstageEmbeddings(
    model = "solar-embedding-1-large"
)

In [14]:
import getpass
import os
from dotenv import load_dotenv


from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

load_dotenv()

pinecone_api_key = os.environ.get("PAK")
os.environ["PINECONE_API_KEY"] = pinecone_api_key

pc = Pinecone(api_key=pinecone_api_key)

index_name = 'tax-upstage-index'
database = PineconeVectorStore.from_documents(documents=document_list, embedding=embedding, index_name=index_name)




/Users/xeroman.k/ws/inflearn-llm-app/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/xeroman.k/ws/inflearn-llm-app/.venv/lib/python3.10/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorSt

In [15]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# retrieved_docs = database.similarity_search(query, 3)

In [16]:
from langchain_upstage import ChatUpstage
llm = ChatUpstage()


In [17]:
# ai_message = llm.invoke(prompt)

In [18]:
# ai_message.content

In [19]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [20]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [24]:
from langchain.chains import RetrievalQA

retriever = database.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [25]:
retriever.invoke(query)

[Document(id='3056bb14-8f98-4369-9931-576c8ec12399', metadata={'source': './tax_docs/tax_with_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n종합소득 과세표준\n\n세율\n\n1,400만원 초과\n\n과세표준의 6퍼센트\n\n1,400만원 초과 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세

In [22]:
ai_message = qa_chain({"query": query})

/var/folders/4b/f3tc2wns1c31fttyw185bb700000gn/T/ipykernel_21888/314547319.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


In [23]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': "연봉 5천만원인 직장인의 소득세는 575만원입니다. 이는 '5,000만원 초과 8,800만원 이하' 세율에 해당하며, 계산식은 '624만원 + (5,000만원을 초과하는 금액인 500만원의 24퍼센트)'입니다."}